# 📊 Data Fetching Pipeline Overview

This notebook demonstrates **three fundamental approaches** to data collection:

1. 🗄️ **Database Queries** - Fetching structured data from MySQL databases
2. 🌐 **API Integration** - Retrieving data through RESTful API endpoints  
3. 🕸️ **Web Scraping** - Extracting data directly from HTML websites

---


## 1️⃣ 🗄️ Fetching Data from Database (MySQL)

### 📦 **Pandas Library**
**Purpose:** Core Python library for data manipulation and analysis  
**Used for:** DataFrame operations, SQL integration, data cleaning  
**Import:** `import pandas as pd`


In [1]:
import pandas as pd

### 🔌 **MySQL Connection Dependencies**
**Installing Required Packages:**
- `mysql-connector` - Official MySQL driver for Python
- `sqlalchemy` - SQL toolkit and ORM
- `pymysql` - Pure Python MySQL client library


In [49]:
!pip install mysql.connector
!pip install sqlalchemy


### 📥 **Import MySQL Connector**
**Library:** `mysql.connector`  
**Purpose:** Enables Python to communicate with MySQL database servers  
**Key Functions:** Establishing connections, executing queries, fetching results


In [50]:
import mysql.connector 

### 🔗 **Establish Database Connection**
**Function:** `mysql.connector.connect()`  
**Parameters:**  
- `host` - Database server address (localhost for local development)
- `user` - MySQL username
- `password` - Authentication password  
- `database` - Target database name

**Returns:** Connection object for executing SQL queries


In [ ]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='world'
)


### 📊 **Execute SQL Query & Load Data**
**Function:** `pd.read_sql_query()`  
**Purpose:** Executes SQL query and returns results as pandas DataFrame  
**Parameters:**  
- SQL query string
- Database connection object

**Use Case:** Fetching filtered data (e.g., US cities from 'city' table)


In [ ]:
pd.read_sql_query("SELECT * FROM city WHERE CountryCode LIKE 'USA'", conn)

---

✅ **Section 1 Complete:** Database Query Mastered!  
⬇️ **Next:** API Data Retrieval

---


---

## 2️⃣ 🌐 Fetching Data From API

### 📦 **Requests Library**
**Purpose:** HTTP library for making API calls  
**Used for:** Sending GET/POST requests to web APIs and handling responses  
**Key Methods:** `get()`, `post()`, `json()`


### 🌐 **Making API Request**
**Process Flow:**
1. 📍 **URL** - API endpoint address
2. 🔑 **Headers** - Authentication keys (x-rapidapi-key, x-rapidapi-host)
3. ⚙️ **Query Parameters** - Search filters, pagination, sorting options
4. 📡 **Send Request** - `requests.get()` with URL, headers, and params
5. 📥 **Parse Response** - Convert JSON response to DataFrame using `response.json()`

**Example:** Fetching anime data from RapidAPI


In [6]:
!pip install pandas

In [3]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY not set")

url = "https://anime-db.p.rapidapi.com/anime"

querystring = {
    "page": "1",
    "size": "10",
    "search": "Fullmetal",
    "genres": "Fantasy,Drama",
    "sortBy": "ranking",
    "sortOrder": "asc"
}

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "anime-db.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
df = pd.DataFrame(response.json()["data"])


In [4]:
print(df.shape)

(9, 18)


In [5]:
df


,_id,id,__v,title,alternativeTitles,genres,image,thumb,link,ranking,synopsis,episodes,status,type,hasRanking,hasEpisode,updatedAt,workerId
0,5114,5114,1.0,Fullmetal Alchemist: Brotherhood,[Hagane no Renkinjutsushi: Fullmetal Alchemist...,"[Action, Adventure, Drama, Fantasy]",https://cdn.myanimelist.net/images/anime/1208/...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/5114/Fullmetal_A...,4,After a horrific alchemy experiment goes wrong...,64,Finished Airing,TV,True,True,None,qk0af
1,121,121,1.0,Fullmetal Alchemist,"[Hagane no Renkinjutsushi, FMA, Full Metal Alc...","[Action, Adventure, Award Winning, Drama, Fant...",https://cdn.myanimelist.net/images/anime/10/75...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/121/Fullmetal_Al...,546,"Edward Elric, a young, brilliant alchemist, ha...",51,Finished Airing,TV,True,True,None,qk0af
2,6421,6421,1.0,Fullmetal Alchemist: Brotherhood Specials,"[Moumoku no Renkinjutsushi, The Blind Alchemis...","[Action, Adventure, Drama, Fantasy]",https://cdn.myanimelist.net/images/anime/1493/...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/6421/Fullmetal_A...,714,1. Moumoku no Renkinjutsushi (The Blind Alchem...,4,Finished Airing,Special,True,True,None,qk0af
3,430,430,NaN,Fullmetal Alchemist: The Conqueror of Shamballa,[Gekijyouban Hagane no Renkinjutsushi - Shanba...,"[Action, Adventure, Award Winning, Drama, Fant...",https://cdn.myanimelist.net/images/anime/1707/...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/430/Fullmetal_Al...,2091,"In desperation, Edward Elric sacrificed his bo...",1,Finished Airing,Movie,True,True,None,qk0af
4,7902,7902,1.0,Fullmetal Alchemist: Brotherhood - 4-koma Gekijou,[Hagane no Renkinjutsushi: Hagaren 4-koma Geki...,"[Comedy, Fantasy]",https://cdn.myanimelist.net/images/anime/3/761...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/7902/Fullmetal_A...,2326,Short specials from the DVDs/BDs.,16,Finished Airing,Special,True,True,None,qk0af
5,908,908,1.0,Fullmetal Alchemist: Premium Collection,"[State Alchemists vs Seven Homunculi, Enkai-he...","[Comedy, Fantasy]",https://cdn.myanimelist.net/images/anime/10/18...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/908/Fullmetal_Al...,3040,1. State Alchemists vs Seven Homunculi\nA 10 m...,3,Finished Airing,OVA,True,True,None,qk0af
6,9135,9135,1.0,Fullmetal Alchemist: The Sacred Star of Milos,"[Fullmetal Alchemist: Milos no Seinaru Hoshi, ...","[Action, Adventure, Drama, Fantasy]",https://cdn.myanimelist.net/images/anime/1217/...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/9135/Fullmetal_A...,3215,Chasing a runaway alchemist with strange power...,1,Finished Airing,Movie,True,True,None,qk0af
7,664,664,1.0,Fullmetal Alchemist: Reflections,"[Hagane no Renkinjutsushi: Tabi no Hajimari, F...","[Adventure, Comedy, Drama, Fantasy]",https://cdn.myanimelist.net/images/anime/2/186...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/664/Fullmetal_Al...,3391,A reflection on what happened during the FMA T...,1,Finished Airing,Special,True,True,None,qk0af
8,10842,10842,1.0,Fullmetal Alchemist: The Sacred Star of Milos ...,[Hagane no Renkinjutsushi: Milos no Seinaru Ho...,"[Comedy, Fantasy]",https://cdn.myanimelist.net/images/anime/9/299...,https://cdn.myanimelist.net/r/50x70/images/ani...,https://myanimelist.net/anime/10842/Fullmetal_...,5670,To mark the July 2 opening of the Fullmetal Al...,4,Finished Airing,ONA,True,True,None,qk0af


---

✅ **Section 2 Complete:** API Integration Done!  
⬇️ **Next:** Web Scraping Techniques

---


---

## 3️⃣ 🕸️ Fetching Data From Web Scraping

### 📦 **Web Scraping Libraries**
**Required Packages:**

**🐍 BeautifulSoup (bs4)**
- HTML/XML parser for extracting data from web pages
- Navigates and searches the parse tree
- Usage: `from bs4 import BeautifulSoup`

**🔧 lxml**
- Fast XML and HTML parser
- Backend parser used by BeautifulSoup
- Better performance for large documents

**🌐 requests**
- HTTP library to fetch webpage HTML content
- Combined with BeautifulSoup for complete scraping workflow


In [8]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import lxml

### 🎭 **User-Agent Header**
**Purpose:** Tells the website we're visiting that this is a browser request  
**Why needed:** Helps avoid "Access Denied" (403) errors  
**Usage:** Pass in `headers` parameter to `requests.get()`


In [9]:
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}

### 📡 **Fetch Webpage HTML**
**Function:** `requests.get()`  
**Purpose:** Fetches raw HTML data from the given URL  
**Returns:** Response object with `.text` attribute containing HTML


In [10]:
webpage =requests.get('https://www.ambitionbox.com/list-of-companies?page=1',headers=headers).text

### 🍜 **Parse HTML with BeautifulSoup**
**Function:** `BeautifulSoup(webpage, 'lxml')`  
**Purpose:** Converts unstructured HTML into structured, searchable tree  
**Parser:** 'lxml' tells BeautifulSoup that data is HTML format  
**Why needed:** Makes it easier to navigate and extract specific data


In [11]:
soup = BeautifulSoup(webpage , 'html.parser')

### 🔍 **Find All Company Cards**
**Function:** `soup.find_all('div', class_='companyCardWrapper')`  
**Purpose:** Finds all company card containers on the page  
**Returns:** List of all matching `<div>` elements  
**Usage:** First step to extract multiple companies from single page


In [12]:
company = soup.find_all('div', class_='companyCardWrapper')
len(company)

20

### 📝 **Extract Company Details**
**Process:** Loop through each company card and extract:
- **Company Name** - Using `find('h2')`
- **Rating** - From `rating_star_container` class
- **Number of Reviews** - From `companyRatingCount` class
- **Company Type & Location** - From `interLinking` class (split by `|`)

**Data Storage:** Lists that will be converted to DataFrame


In [13]:
names=[]
rating=[]
No_Of_Reviews=[]
ctype=[]
locations=[]
company_type=''
location=''

for i in company:
    names.append(i.find('h2').text.strip())
    rating.append(i.find('div', class_='rating_star_container').text.strip())
    No_Of_Reviews.append(i.find('span' , class_='companyCardWrapper__companyRatingCount').text.strip())
    misc_info = i.find('span', class_='companyCardWrapper__interLinking')
    if misc_info:
    # Split by "|" to separate type and location
        parts = misc_info.text.split('|')
        
        # Company Type (first part)
        company_type = parts[0].strip() if len(parts) > 0 else ''
        ctype.append(company_type)
        
        # Location (second part)
        location = parts[1].strip() if len(parts) > 1 else ''
        locations.append(location)
    else:
        ctype.append('')
        locations.append('')
    

In [14]:
names

['TCS',
 'Accenture',
 'Wipro',
 'Cognizant',
 'Capgemini',
 'HDFC Bank',
 'Infosys',
 'ICICI Bank',
 'HCLTech',
 'Tech Mahindra',
 'Genpact',
 'Teleperformance',
 'Jio',
 'Axis Bank',
 'Concentrix Corporation',
 'Amazon',
 'iEnergizer',
 'Reliance Retail',
 'LTIMindtree',
 'IBM']

In [15]:
rating

['3.3',
 '3.7',
 '3.6',
 '3.6',
 '3.7',
 '3.8',
 '3.5',
 '4.0',
 '3.4',
 '3.4',
 '3.6',
 '3.8',
 '4.4',
 '3.6',
 '3.6',
 '3.9',
 '4.6',
 '3.9',
 '3.6',
 '3.9']

In [16]:
No_Of_Reviews

['(1.1L)',
 '(71.4k)',
 '(63.5k)',
 '(59.9k)',
 '(51.5k)',
 '(50.7k)',
 '(47.3k)',
 '(45.1k)',
 '(44.5k)',
 '(42.4k)',
 '(40.9k)',
 '(36.6k)',
 '(32.5k)',
 '(32.4k)',
 '(31.5k)',
 '(30.7k)',
 '(27.1k)',
 '(27k)',
 '(25.8k)',
 '(25.3k)']

In [17]:
ctype

['IT Services & Consulting',
 'IT Services & Consulting',
 'IT Services & Consulting',
 'IT Services & Consulting',
 'IT Services & Consulting',
 'Banking',
 'IT Services & Consulting',
 'Banking',
 'IT Services & Consulting',
 'IT Services & Consulting',
 'IT Services & Consulting',
 'BPO',
 'Telecom',
 'Banking',
 'BPO',
 'Internet',
 'BPO',
 'Retail',
 'IT Services & Consulting',
 'IT Services & Consulting']

In [18]:
locations

['Bangalore / Bengaluru +439 other locations',
 'Bangalore / Bengaluru +255 other locations',
 'Hyderabad / Secunderabad +370 other locations',
 'Hyderabad / Secunderabad +230 other locations',
 'Bangalore / Bengaluru +183 other locations',
 'Mumbai +1824 other locations',
 'Bangalore / Bengaluru +247 other locations',
 'Mumbai +1437 other locations',
 'Chennai +228 other locations',
 'Hyderabad / Secunderabad +332 other locations',
 'Hyderabad / Secunderabad +181 other locations',
 'Mumbai +255 other locations',
 'Mumbai +1911 other locations',
 'Mumbai +1503 other locations',
 'Bangalore / Bengaluru +176 other locations',
 'Bangalore / Bengaluru +519 other locations',
 'Noida +51 other locations',
 'Mumbai +1156 other locations',
 'Bangalore / Bengaluru +145 other locations',
 'Bangalore / Bengaluru +160 other locations']

### 📊 **Create DataFrame from Lists**
**Function:** `pd.DataFrame()`  
**Purpose:** Converts extracted lists into structured tabular data  
**Columns Created:**
- `Company_Name` - Company name
- `Rating` - Rating score
- `No_Of_Reviews` - Review count
- `Company_Type` - Business category
- `Location` - Office locations


In [19]:
df = pd.DataFrame({
    'Company_Name': names,
    'Rating': rating,
    'No_Of_Reviews': No_Of_Reviews,
    'Company_Type': ctype,
    'Location': locations
})


In [20]:
df

,Company_Name,Rating,No_Of_Reviews,Company_Type,Location
0,TCS,3.3,(1.1L),IT Services & Consulting,Bangalore / Bengaluru +439 other locations
1,Accenture,3.7,(71.3k),IT Services & Consulting,Bangalore / Bengaluru +255 other locations
2,Wipro,3.6,(63.5k),IT Services & Consulting,Hyderabad / Secunderabad +370 other locations
3,Cognizant,3.6,(59.8k),IT Services & Consulting,Hyderabad / Secunderabad +229 other locations
4,Capgemini,3.7,(51.4k),IT Services & Consulting,Bangalore / Bengaluru +183 other locations
5,HDFC Bank,3.8,(50.6k),Banking,Mumbai +1823 other locations
6,Infosys,3.5,(47.3k),IT Services & Consulting,Bangalore / Bengaluru +246 other locations
7,ICICI Bank,4.0,(45.1k),Banking,Mumbai +1437 other locations
8,HCLTech,3.4,(44.5k),IT Services & Consulting,Chennai +228 other locations
9,Tech Mahindra,3.4,(42.4k),IT Services & Consulting,Hyderabad / Secunderabad +332 other locations


---

## 🔁 **Scaling Up: Multiple Pages**

### 🔄 **Loop Through All Pages**
**Strategy:** Iterate from page 1 to page 330 to collect all company data  
**Process:**
1. Generate URL for each page
2. Fetch and parse HTML
3. Extract company data
4. Append to combined list
5. Convert to final DataFrame


In [24]:
def safe_text(parent, tag, class_name=None):
    el = parent.find(tag, class_=class_name)
    return el.text.strip() if el else None


In [28]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup



final = pd.DataFrame()

for j in range (1,501):
    url='https://www.ambitionbox.com/list-of-companies?page={}'.format(j)
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    webpage =requests.get(url,headers=headers).text
    soup = BeautifulSoup(webpage , 'html.parser')
    company = soup.find_all('div', class_='companyCardWrapper')
   
    names=[]
    rating=[]
    No_Of_Reviews=[]
    ctype=[]
    locations=[]
    company_type=''
    location=''
    for i in company:
            names.append(safe_text(i, 'h2','companyCardWrapper__companyName'))
            rating.append(safe_text(i, 'div', 'rating_star_container'))
            No_Of_Reviews.append(
                safe_text(i, 'span', 'companyCardWrapper__companyRatingCount')
            )
        
            misc_info = i.find('span', class_='companyCardWrapper__interLinking')
            if misc_info:
                parts = misc_info.text.split('|')
                ctype.append(parts[0].strip() if len(parts) > 0 else None)
                locations.append(parts[1].strip() if len(parts) > 1 else None)
            else:
                ctype.append(None)
                locations.append(None)

    df = pd.DataFrame({
        'Company_Name': names,
        'Rating': rating,
        'No_Of_Reviews': No_Of_Reviews,
        'Company_Type': ctype,
        'Location': locations
    })

    final = pd.concat([final, df], ignore_index=True)

                    
            
    


In [29]:
final


,Company_Name,Rating,No_Of_Reviews,Company_Type,Location
0,TCS,3.3,(1.1L),IT Services & Consulting,Bangalore / Bengaluru +439 other locations
1,Accenture,3.7,(71.4k),IT Services & Consulting,Bangalore / Bengaluru +255 other locations
2,Wipro,3.6,(63.5k),IT Services & Consulting,Hyderabad / Secunderabad +370 other locations
3,Cognizant,3.6,(59.9k),IT Services & Consulting,Hyderabad / Secunderabad +230 other locations
4,Capgemini,3.7,(51.5k),IT Services & Consulting,Bangalore / Bengaluru +183 other locations
...,...,...,...,...,...
9995,Emerald Haven Realty,3.7,(106),Real Estate,Chennai +4 other locations
9996,Fedders Lloyd,3.5,(106),Engineering & Construction,New Delhi +25 other locations
9997,SMC Real Estate Advisors,3.0,(106),Real Estate,Pune +11 other locations
9998,GRG Health,2.8,(106),Pune +3 other locations,None


In [30]:
df

,Company_Name,Rating,No_Of_Reviews,Company_Type,Location
0,Chopra Industries,3.4,(106),Auto Components,New Delhi +24 other locations
1,Chassis Brakes International,3.3,(106),Auto Components,Jalgaon +8 other locations
2,Lazza Ice Creams,3.0,(106),Food Processing,Kochi +24 other locations
3,Kisan Mouldings,4.2,(106),Building Material,Mumbai +29 other locations
4,Anonymous Content,4.1,(106),Pune +19 other locations,None
5,Action for Social Advancement,3.4,(106),Non-Profit,Bhopal +28 other locations
6,CTA Apparels,3.6,(106),Fashion & Textile,Noida +7 other locations
7,SRIT,3.6,(106),IT Services & Consulting,Bangalore / Bengaluru +20 other locations
8,Jal International,3.8,(106),Oil & Gas,Chennai +12 other locations
9,Life Style Foods,3.6,(106),Food Processing,New Delhi +15 other locations


### 💾 **Save Data to CSV**
**Function:** `final.to_csv('ambitionbox_companies1.csv')`  
**Purpose:** Exports DataFrame to CSV file for future use  
**Output:** CSV file with 10,000+ company records  
**Use Cases:** Data analysis, ML models, business intelligence


In [ ]:
final.to_csv('ambitionbox_companies1.csv')

---

## 🎉 **Congratulations!**

You've learned **three powerful data collection methods**:

✅ **Database Queries** - Structured data from SQL databases  
✅ **API Integration** - JSON data from RESTful APIs  
✅ **Web Scraping** - Data extraction from HTML websites

### 🚀 **Next Steps:**
- Data Cleaning & Preprocessing
- Exploratory Data Analysis (EDA)
- Feature Engineering
- Machine Learning Model Building

---

**Happy Data Science! 📊🐍**
